# **Ejercicios de pair programming Módulo 2 Sprint 2**
## **Limpieza III**
### **Valores Extremos**

Hipótesis

La edad, el trabajo, el estado civil, la educación, la situación de deuda y la forma de contacto pueden influir en la probabilidad de que un cliente acepte la oferta.

El número de veces que se ha contactado a un cliente en el pasado (campo campaign), el número de días que han pasado desde el último contacto (campo pdays), y el resultado de la campaña anterior (campo poutcome) pueden afectar la respuesta del cliente a una nueva oferta.

Las variables económicas (tales como el índice de precios al consumidor (cons.price.idx), la tasa de variación del empleo (emp.var.rate), etc.) pueden influir en la probabilidad de que un cliente acepte la oferta.

Los clientes que ya tienen una hipoteca (housing) o un préstamo (loan) pueden ser menos propensos a aceptar una nueva oferta, ya que podrían estar limitados financieramente.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sidetable

plt.rcParams["figure.figsize"] = (10,8)
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv('../datos/df_bank_clean', index_col = 0) ### comentar si abrimos este o el 2

Ya hemos limpiado todas las columnas que parecía que no tenían el tipo de dato que queríamos o que tenían demasiada información. Por lo tanto, en este ejercicio tendremos que limpiar los outliers.

¿Sobre qué columnas podremos eliminar o reemplazar los outliers?

Identifica visualmente los outliers

Identifica sobre el dataframe las filas que son outliers

Reemplaza los outliers por el estadístico más correcto.

Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio en el siguiente pair programming de limpieza.